# COVID Forecast Modelling

When COVID presented a new challenge for modelling, the models used for forecasting developed. Traditionally, an SIR model has been used. Advancing introduced the SEIR model. Even then, it was difficult to forecast the effects of COVID. There have been further developments in this area that use agent-based models to reinact the behaviours and profiles of people that will affect their exposure, their chances of infection, their chances of asymptomatic presentation of the virus and of course, the recovery outcome. This problem is not a simple one when we consider the variations in physiology and their environment, including not only factors such as population density but also cultural differences and the decisions made by the world leaders. For more information about the complicated nature of forecasting COVID, this [article](https://media.nature.com/original/magazine-assets/d41586-020-01003-6/d41586-020-01003-6.pdf) gives a high-level overview of the challenges and trails that were experienced by professionals modelling the virus. 

In order to tackle this challenge, I have chosen to develop on a prepared model whose author [Boris Tseytlin](https://github.com/btseytlin) is experienced in the area with [publications](https://arxiv.org/abs/2106.08048) to support their work.  Data in this model was taken from a freely available [source](https://yandex.ru/covid19/stat) which gives values for COVID related information from Moscow. 

The parameters, input values and assumptions used to build this model are based on real-world information from Moscow. We seen that during COVID, each world leader made decisions to handle and reduce the effects of the virus on their population. 

We can use the code here to evaluate how well fitted this model would be on data from other countries. Given that his model forecasts not only the visible deaths and infections, but the invisible; this gives a more real-world alternative forecasting that may be more indicative of true numbers that were seen during the pandemic.

Included in this analysis were the following countries:
- Thailand
- Hong Kong
- Taiwan
- South Korea

These countries were selected as they contained the most consistent data points over the span of time provided in the data.

The following chinese states were included in order to represent the models performance within a country that are less likely to be affected by cultural differences that may be present between countries. Since we want to focus our study on incorporating the population size of a state or country, we keep Mainland China states to test size between states consistent with a government who took a blanket approach to COVID precautions. It was difficult to find data for the chinese states, but as the following selected are popular states within mainland China, we were able to obtain the figures.
- Gansu
- Inner Mongolia
- Qinghai
- Tibet
- Xinjiang

## <span style="color:blue"> **Data Preparation**</span>

`notebooks/data_preparation_countries`

The first step was to perform <span style="color:green"> **EDA**</span>

- **Duplication and erroneous entry of Country names:** Taking a look at the countries that were included in the data, it was found that there are some duplicate countries that have been entered with inconsistent names that do not match.
- **Observation Frequency:** Dates running from 01/22/2020 to 08/29/2020, tell us that there is a total 221 days of recorded data. As our first run through, we will concentrate on countries that were able to consistently provide observations for each day.
- **Dates reliability:** Comparing the two possible date columns shows that nearly 85000 dates do not match between Last Update and ObservationDate which isn't necessarily a problem depending on the definition of the data in the columns. <span style="color:red"> ASSUMPTION!: </span> If we take a look at the data values that aren't matching, we can see that the dates of Last Updates can either be before or after the ObservationDate. Assuming that the 'LastUpdate' refers to the datetime of any initial entries, updates or amendments to the data values, it would appear odd that values for any given ObservationDate would be submitted before the date had passed.
- **Observation Frequency by Country/ State:** There were significant differences between the number of observations for each country. From this we could see that some countries had submitted data for the entire country, while some countries had submitted data for sub areas of the countries. We do not see consistent result entries for states within the US. Japan's entries are consistent across the states however, we only seem to have 94 observation dates. Trying a few states, we can see that the data for the Japanese states only covers a small span of time which is consistent through may to August


Since there is some consistency issues with some of the countries submissions, we decided to remove them since we've already decided to work only with countries that have ObservationDates for each date between 01/22/2020 to 08/29/2020.


<span style="color:green"> **Additional data sources**</span>

Next, we gathered additional data sources to included population size into the model. Using a subset of data available at the URL below, we curated a dataset of information that may be interesting to our work https://population.un.org/wpp/Download/Standard/CSV/. 

Within this new dataset, we managed to find Thailand with a simple match, Taiwan and Hong Kong required a fuzzy match using regex best match algorithm. However, we were still missing South Korea and the chinese states (even if we increase the error rate threshold). String approximation methods and NLP techniques can be used to enhance the search (such Levenshtein, Mongue-Elkan, Dependency Parsing, etc.) but for this particular project, it isn't worth spending the time to create a function for it. So let's just go and find the data. A little research and we find that South Korea can also be referred to as the 'Republic of Korea' which matches the entry in our new dataset.

The chinese states were harder to get population statistics for since the data source we were using only contained population statistics for countries. Since we couldn't find a reputable source (without paying for it) which listed all of the chinese states, we've taken data from Wikipedia for the states that we could find (Source: National Bureau of Statistics) :: https://en.wikipedia.org/wiki/Demographics_of_China#Population_density_and_distribution


<span style="color:green"> **Smoothing and Merging**</span>

Next we smooth our data by creating a column that captures the rolling mean across a 7 day period. We creates a column containing the cumulative sum of the data points over time to capture the tally as time went on. Next, we merged our dataframes and renamed our columns. Finally pickling our file for use in the model.

## <span style="color:blue"> **Model Results**</span>

In order to test the countries in the model, the jupyter notebook implementation (`notebooks/seir_moscow_hidden.ipynb`) was adapted into a python file (`main.py`) which could run the results iteratively. Changing the code so that the population size could be fed in.

The main result that we are interested in is the error rate on the model:

| Country / State | Population Size (thousands) | Population Density (km2) | Baseline Error Rate | Model Error Rate |
| --- | --- | --- | --- | --- |
| Gansu | 26172.0 | 57 | 30.7 | 1688.177 |
| Inner Mongolia | 24051.0 | 20 | 15.254 | 86.894 |
| Quinghai | 5516.0 | 7 |  0.0 | 37.757 |
| Tibet | 2842.0 | 2 |  0.0 | 0.973 |
| Xinjiang | 20952.0 | 12 |  46.5 | 1001.638 |
| Hong Kong | 7500.958 | 6825 |  113.558 | 906.098 |
| Thailand | 71475.664 | 139 |  671.654 | 1083.803 |
| Taiwan | 23821.464 | 672 |  87.508 | 206.43 |
| South Korea | 51844.69 | 523 |  3562.5 | 11679.954 |

Taking 10 dates to evaluate:

**Baseline:** Starting at the evaluation date, the number of total deaths is repeated across the 30 days.

**Model:** Forecast prediction of number of deaths for the 30 day period, starting at the eval date

Comparing both of these to the true deaths using MAE, we arrive at our error rate. We can see from the data in the table that the model was unable to perform better than the baseline in all cases. This may be due to the quaratine multiplier based on Russian / Moscow lockdown laws. However, we can still compare the rates between countries and how well the model performed.

### Summary of results:
- For models when the baseline was low to 0, if we refer to the plots produced in the Data preparation stage (`notebooks/data_preparation_countries`), we can see that the number of deaths started to plateau in Mainland China (which covers all chinese states). This is a suggestion that the strict lockdown precautions in Mainland China were effective in reducing the spread of the virus.
- Comparably, we see that the baseline of Taiwan was also low, given the high population density which seems to correspond to higher baseline and model error rates, this seems anomolous, especially when referencing to the lockdown measures which were more relaxed than the other countries / states being compared here.
> [Lockdown Restrictions: Taiwan](https://graphics.reuters.com/world-coronavirus-tracker-and-maps/countries-and-territories/taiwan/)

> [Lockdown Restrictions: South Korea](https://graphics.reuters.com/world-coronavirus-tracker-and-maps/countries-and-territories/south-korea/)

> [Lockdown Restrictions: Hong Kong](https://graphics.reuters.com/world-coronavirus-tracker-and-maps/countries-and-territories/hong-kong//)

> [Lockdown Restrictions: Thailand](https://graphics.reuters.com/world-coronavirus-tracker-and-maps/countries-and-territories/thailand/)

> [Lockdown Restrictions: Chine](https://graphics.reuters.com/world-coronavirus-tracker-and-maps/countries-and-territories/china/)

- Despite being given only population size to amend its' calculations, the model still performed well on many of the countries / states.

For more results, please see `figures` which lists all countries with their visualisations, log and error rate.
